In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
df = spark.read.csv(
    "S2GC.tsv", sep=r'\t', header=False, mode="DROPMALFORMED")

dataColl=df.collect()
for row in dataColl[0:5]:
    print(row[0] + "," +str(row[1]))

S0144,okay what's the problem
S0024,right so well i be i'm just a bite confuse really
S0144,with what
S0024,right you've get to compare the examples of the academic text feature here yeah with this text
S0144,yeah


In [3]:
def extract_speakers(df1):
    dataC = df1.collect()
    speaker = [dataC[0][0]]
    for row in dataC:
        if row[0]!= speaker[0]:
            speaker.append(row[0])
            break
    #rdd1 = rdd1.rdd
    return df1, speaker

In [4]:
def extract_dialogues_dic(rdd1, speaker):
    speaker1 = speaker[0]
    speaker2 = speaker[1]
    speaker1_dialogue = {}
    speaker2_dialogue = {}
    dataC = rdd1.collect()
    for i in range(0,len(dataC)):
        if dataC[i][0] == speaker1:
            if dataC[i][1]!=None:
                speaker1_dialogue[i] = dataC[i][1].split()
#             speaker1_dialogue.append(dataC[i][1].split())
        elif dataC[i][0] == speaker2:
            if dataC[i][1]!=None:
                speaker2_dialogue[i] = dataC[i][1].split()
#             speaker2_dialogue.append(dataC[i][1].split())
    
    return speaker, speaker1_dialogue, speaker2_dialogue

In [5]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
print(stops)

{'out', 'we', 'was', 'how', 'myself', 'for', 'they', 'about', "hasn't", 'then', 'both', 'so', 're', 'don', 'm', 'as', 'any', 'mightn', 'after', 'you', 'wouldn', 'why', 'been', 'where', 'by', "isn't", 'yourself', 'wasn', 'a', "haven't", 'did', "hadn't", 'their', 'hasn', 'doing', 'be', 'further', 'ours', 'now', 'am', 'her', "you'll", 'yourselves', 'that', 'my', 'what', 'to', 'd', 'not', "won't", "couldn't", 'own', 'there', 'this', 'each', 'all', 'haven', 'more', 'me', 've', 'weren', 'which', 'himself', 'nor', 'other', "shouldn't", 'who', "should've", 'same', 'at', 'such', 't', 'up', 'than', 'can', "you've", 'too', 'these', 'while', "wasn't", 'ourselves', 'before', 'i', 'he', "didn't", 'our', 'its', 'but', 'with', "wouldn't", 'those', 'because', 'the', 'y', 'shouldn', 'it', 'mustn', 'hers', 'just', 'doesn', 'ain', 'between', 'over', 'had', 'aren', "mightn't", 'does', 'have', 'and', 'or', 'some', "mustn't", 'only', 'won', 'when', 'needn', 'below', 'in', 'if', 'theirs', "needn't", "aren't",

In [6]:
def get_er_one_with_u(x,y, exclude_stops):
# need to be changed in other files
    record=[[0 for i in range(len(y)+1)]  for j in range(len(x)+1)]
    dic_record_s1 = {}
    dic_record_s2 = {}
    expression_list_1 = []
    expression_list_2 = []
    expression_list_1_token = []
    expression_list_2_token = []
    for i in range(0,len(x)):
        for j in range(0,len(y)):
            if x[i] == y[j]:
                record[i+1][j+1] = record[i][j] + 1
                if record[i+1][j+1] > 0:
                        if i not in dic_record_s1.keys():
                            dic_record_s1[i] = record[i+1][j+1]
                        elif i in dic_record_s1.keys():
                            if (dic_record_s1[i]<record[i+1][j+1]):
                                dic_record_s1[i] = record[i+1][j+1]
                                
                        if j not in dic_record_s2.keys():
                            dic_record_s2[j] = record[i+1][j+1]
                        elif j in dic_record_s2.keys():
                            if (dic_record_s2[j]<record[i+1][j+1]):
                                dic_record_s2[j] = record[i+1][j+1]

    if dic_record_s1== {}:
        return []
    
    else:
        i = max(dic_record_s1.values())
        
        
    if dic_record_s2== {}:
        return []
    
    else:
        j = max(dic_record_s2.values())
    
    while i>0:  
        for key in dic_record_s1:
            if dic_record_s1[key] == i:
            #expression_repitition_record = expression_repitition_record+1
                expression = x[key+1-i : key+1]
                expression_string = ' '.join(expression)
            
                for k in range(0,i):
                    dic_record_s1[key - k] = 0
            
#             if expression_list != []:
#                 for e in expression_list:
#                     if expression not in e:
#                         expression_list.append(expression) 
#             else:

#                 if expression_list == []:
#                     expression_list.append(expression_string)
#                 else:
#                     for ex in expression_list:
#                         if expression string not in ex:
#                             expression_list.append(expression)
                if exclude_stops == True:
                    if expression_string not in stops:
                        if expression_string not in expression_list_1:
                            expression_list_1.append(expression_string)
                            expression_list_1_token.append(expression)
                                     
                else:
                    if expression_string not in expression_list_1:
                        expression_list_1.append(expression_string)
#                 lens1 = lens1 + len(expression)
                        expression_list_1_token.append(expression)
        
        i = i-1
        
    while j>0:  
        for key in dic_record_s2:
            if dic_record_s2[key] == j:
            #expression_repitition_record = expression_repitition_record+1
                expression_2 = y[key+1-j : key+1]
                expression_string_2 = ' '.join(expression_2)
            
                for k in range(0,j):
                    dic_record_s2[key - k] = 0
                
                
                
                if exclude_stops == True:
                    if expression_string_2 not in stops:
                        if expression_string_2 not in expression_list_1:
                            expression_list_1.append(expression_string_2)
                            expression_list_1_token.append(expression_2)
                
                else:
                    if expression_string_2 not in expression_list_1:
                        expression_list_1.append(expression_string_2)
                        expression_list_1_token.append(expression_2)
        
        j = j-1
        
    return expression_list_1

In [7]:
def get_er_dic(speaker, speaker1_dialogue, speaker2_dialogue):
    expressions = {}
    
    for x,value_x in speaker1_dialogue.items():
        for y,value_y in speaker2_dialogue.items():
            string = get_er_one_with_u(value_x, value_y, False)
            value_x_sentence = ' '.join(value_x)
            value_y_sentence = ' '.join(value_y)
            
            for s in string:
                
                if s not in expressions.keys():
                    
                    pos = max(x,y)
                    expressions[s] = [pos,[x],[y]]
                    
                elif s in expressions.keys():
                    if (x in expressions[s][1]) and (y not in expressions[s][2]):
                        expressions[s][2].append(y)
                        expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
                    elif (y in expressions[s][2]) and (x not in expressions[s][1]):
                        expressions[s][1].append(x)
                        expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
#                     elif (y in expressions[s][2]) and (x in expressions[s][1]):
#                         expressions[s][0] = expressions[s][0]+count1+count2
                    
#                     expressions[s][1].append(x)
#                     expressions[s][2].append(y)
    
    return expressions

In [8]:
def ex_in_timeslot(expression_dic,u_slot):
    ex_in_time = {}
    for k, v in expression_dic.items():
        flag = True
        for x in v[1]:
            for y in v[2]:
                if (v[0] <= u_slot) and (x < u_slot) and (y < u_slot):
                    if k not in ex_in_time.keys():
                        ex_in_time[k] = v
                        flag = False
                        break
            if not flag:
                break
    return ex_in_time  

In [9]:
def get_er_ratio_in_time_before(speaker, speaker1_dialogue, speaker2_dialogue, ex_in_u, u_low, u_high):
    max_len = 0
    for key in ex_in_u.keys():
        max_len = max(len(key.split()),max_len)
    
    i = max_len
    dialogue_s1 = {}
    dialogue_s2 = {}
    count_u1 = 0
    count_u2 = 0
    count_u1_token = 0
    count_u2_token = 0
    token_sum_s1 = 0
    token_sum_s2 = 0
    
    for k,v in speaker1_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s1 = token_sum_s1 + len(v)
            dialogue_s1[k] = ' '.join(speaker1_dialogue[k])
            
    
    for k,v in speaker2_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s2 = token_sum_s2 + len(v)
            dialogue_s2[k] = ' '.join(speaker2_dialogue[k])
        
    while i>1:
        for key,value in ex_in_u.items():
            if len(key.split()) == i:
                for u1 in value[1]:
                    if (u1>=u_low) and (u1<u_high) and (u1>= value[0]):
#                         print(u1,key)
#                 dialogue_dic_s1[u1] = ' '.join(dialogue_dic_s1[u1])
                        c1 = dialogue_s1[u1].count(key)
                        dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
                        count_u1 = c1+count_u1
                        count_u1_token = i*c1 + count_u1_token
                for u2 in value[2]:
                    if (u2>=u_low) and (u2<u_high) and (u2>= value[0]):
#                         print(u2,key)
#                 dialogue_dic_s2[u2] = ' '.join(dialogue_dic_s2[u2])
                        c2 = dialogue_s2[u2].count(key)
                        dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                        count_u2 = c2+count_u2
                        count_u2_token = i*c2 + count_u2_token
    
        i = i-1
        
    for key,value in ex_in_u.items():
        if len(key.split()) == 1:
            for u1 in value[1]:
                if (u1>=u_low) and (u1<u_high) and (u1>= value[0]):
#                         print(u1,key)
#                 dialogue_dic_s1[u1] = ' '.join(dialogue_dic_s1[u1])
                    c1 = dialogue_s1[u1].split().count(key)
    
#                     dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
#                 print(dialogue_dic_s1[u1])
                    count_u1 = c1+count_u1
                    count_u1_token = i*c1 + count_u1_token
            for u2 in value[2]:
                if (u2>=u_low) and (u2<u_high) and (u2>= value[0]):
#                         print(u2,key)
#                 dialogue_dic_s2[u2] = ' '.join(dialogue_dic_s2[u2])
                    c2 = dialogue_s2[u2].split().count(key)
#                     dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                    count_u2 = c2+count_u2
                    count_u2_token = i*c2 + count_u2_token
    
             
        
    if token_sum_s1 != 0:
        speaker1_ratio = count_u1_token/token_sum_s1
    else:
        speaker1_ratio = 0
    if token_sum_s2 != 0:
        speaker2_ratio = count_u2_token/token_sum_s2
    else:
        speaker2_ratio = 0
            
    if (token_sum_s2+token_sum_s1) != 0:
        er_ratio = (count_u1_token+count_u2_token)/(token_sum_s1+token_sum_s2)
    else:
        er_ratio = 0
    
    return speaker1_ratio,speaker2_ratio,er_ratio

In [10]:
import copy
import pandas as pd
def get_er_ratio_in_time(speaker, speaker1_dialogue, speaker2_dialogue, ex_in_u, u_low, u_high):
    max_len = 0
    for key in ex_in_u.keys():
        max_len = max(len(key.split()),max_len)
    
    i = max_len
    dialogue_s1 = {}
    dialogue_s2 = {}
    count_u1 = 0
    count_u2 = 0
    count_u1_token = 0
    count_u2_token = 0
    token_sum_s1 = 0
    token_sum_s2 = 0
#     printout =[]
    
    for k,v in speaker1_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s1 = token_sum_s1 + len(v)
            dialogue_s1[k] = ' '.join(speaker1_dialogue[k])
            
    
    for k,v in speaker2_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s2 = token_sum_s2 + len(v)
            dialogue_s2[k] = ' '.join(speaker2_dialogue[k])
        
    while i>1:
        for key,value in ex_in_u.items():
            if len(key.split()) == i:
                for u1 in value[1]:
                    if (u1>=u_low) and (u1<u_high):
                        c1 = dialogue_s1[u1].count(key)
                        dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
                        if (u1>= value[0]):                        
                            count_u1 = c1+count_u1
                            count_u1_token = len(key.split())*c1 + count_u1_token
#                             dic_out = {'key':key, "u":u1, "count_token":c1, "speaker":speaker[0]}
#                             printout.append(dic_out)
                for u2 in value[2]:
                    if (u2>=u_low) and (u2<u_high):
                        c2 = dialogue_s2[u2].count(key)
                        dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                        if (u2>= value[0]):
                            count_u2 = c2+count_u2
                            count_u2_token = len(key.split())*c2 + count_u2_token
#                             dic_out = {'key':key, "u":u2, "count_token":c2, "speaker":speaker[1]}
#                             printout.append(dic_out)
    
        i = i-1
    
    for key,value in ex_in_u.items():
            if len(key.split()) == 1:
                for u1 in value[1]:
                    if (u1>=u_low) and (u1<u_high):
                        c1 = dialogue_s1[u1].split().count(key)
                        if (u1>= value[0]):
                            count_u1 = c1+count_u1
                            count_u1_token = c1 + count_u1_token
#                             dic_out = {'key':key, "u":u1, "count_token":c1, "speaker":speaker[0]}
#                             printout.append(dic_out)

                        
                for u2 in value[2]:
                    if (u2>=u_low) and (u2<u_high) :
                        c2 = dialogue_s2[u2].split().count(key)
                        if (u2>= value[0]):
                            count_u2 = c2+count_u2
                            count_u2_token = c2 + count_u2_token
#                             dic_out = {'key':key, "u":u2, "count_token":c2, "speaker":speaker[1]}
#                             printout.append(dic_out)
    
#     dl_test = pd.DataFrame(printout)
#     dl_test.to_csv(('test_result_s2a5.csv'), index = False)
    
        
    if token_sum_s1 != 0:
        speaker1_ratio = count_u1_token/token_sum_s1
    else:
        speaker1_ratio = 0
    if token_sum_s2 != 0:
        speaker2_ratio = count_u2_token/token_sum_s2
    else:
        speaker2_ratio = 0
            
    if (token_sum_s2+token_sum_s1) != 0:
        er_ratio = (count_u1_token+count_u2_token)/(token_sum_s1+token_sum_s2)
    else:
        er_ratio = 0
    
    return speaker1_ratio,speaker2_ratio,er_ratio

In [11]:
def get_er_ratio_in_time_after(speaker, speaker1_dialogue, speaker2_dialogue, ex_in_u, u_low, u_high):
    max_len = 0
    for key in ex_in_u.keys():
        max_len = max(len(key.split()),max_len)
    
    i = max_len
    dialogue_s1 = {}
    dialogue_s2 = {}
    count_u1 = 0
    count_u2 = 0
    count_u1_token = 0
    count_u2_token = 0
    token_sum_s1 = 0
    token_sum_s2 = 0
    
    for k,v in speaker1_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s1 = token_sum_s1 + len(v)
            dialogue_s1[k] = ' '.join(speaker1_dialogue[k])
            
    
    for k,v in speaker2_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s2 = token_sum_s2 + len(v)
            dialogue_s2[k] = ' '.join(speaker2_dialogue[k])
        
    while i>1:
        for key,value in ex_in_u.items():
            if len(key.split()) == i:
                for u1 in value[1]:
                    if (u1>=u_low and u1<u_high):
#                         print(u1,key)
#                 dialogue_dic_s1[u1] = ' '.join(dialogue_dic_s1[u1])
                        c1 = dialogue_s1[u1].count(' ' + key)
                        dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
#                 print(dialogue_dic_s1[u1])
                        count_u1 = c1+count_u1
                        count_u1_token = i*c1 + count_u1_token
                for u2 in value[2]:
                    if (u2>=u_low and u2<u_high):
#                         print(u2,key)
#                 dialogue_dic_s2[u2] = ' '.join(dialogue_dic_s2[u2])
                        c2 = dialogue_s2[u2].count(' ' + key)
                        dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                        count_u2 = c2+count_u2
                        count_u2_token = i*c2 + count_u2_token
    
        i = i-1
        
      
        if token_sum_s1 != 0:
            speaker1_ratio = count_u1_token/token_sum_s1
        else:
            speaker1_ratio = 0
    
        if token_sum_s2 != 0:
            speaker2_ratio = count_u2_token/token_sum_s2
        else:
            speaker2_ratio = 0
            
        if (token_sum_s2+token_sum_s1) != 0:
            er_ratio = (count_u1_token+count_u2_token)/(token_sum_s1+token_sum_s2)
        else:
            er_ratio = 0
    
    return speaker1_ratio,speaker2_ratio,er_ratio

In [12]:
def number_of_slot(dialogue1,dialogue2,length_u):
    number_of_u = len(dialogue1)+len(dialogue2)
    number_slot = int(number_of_u - number_of_u%length_u)/length_u

    return int(number_slot)

In [13]:
def get_er_ratio_temporal(speaker, speaker1_dialogue, speaker2_dialogue, length, ex_dic_x):
    slot_number = number_of_slot(speaker1_dialogue, speaker2_dialogue,length)
    er_dic = {}
    er_dic_list = []
    
    for i in range(0,slot_number):
        er = get_er_ratio_in_time(speaker,speaker1_dialogue,speaker2_dialogue,ex_in_timeslot(ex_dic_x,length*(i+1)),length*i,length*(i+1))
        er_dic = {'begin':length*i, 'end': length*(i+1), 's1_er_ratio':er[0],'s2_er_ratio':er[1],'er_ratio_average':er[2]}
        er_dic_list.append(er_dic)
    
    er_end = get_er_ratio_in_time(speaker,speaker1_dialogue,speaker2_dialogue,ex_in_timeslot(ex_dic_x,(len(speaker1_dialogue)+len(speaker2_dialogue))),slot_number*length,len(speaker1_dialogue)+len(speaker2_dialogue))
    er_dic_end = {'begin':slot_number*length, 'end': len(speaker1_dialogue)+len(speaker2_dialogue), 's1_er_ratio':er_end[0], 's2_er_ratio':er_end[1], 'er_ratio_average':er_end[2]}  
    er_dic_list.append(er_dic_end)
    
    return er_dic_list
        

In [14]:
speaker = extract_speakers(df)
dialogue_dic = extract_dialogues_dic(speaker[0],speaker[1])

In [15]:
ex_dic = get_er_dic(dialogue_dic[0],dialogue_dic[1],dialogue_dic[2])

In [16]:
print(ex_dic)

{'the': [3, [0, 26, 44, 60, 62, 68, 70, 72, 78, 82, 96, 98, 114, 128, 138, 140, 155, 159, 161, 165, 167, 179, 183, 189, 193, 195, 205, 206, 222, 224, 240, 244, 246, 274, 282, 284, 286, 298, 300, 302, 306, 309, 311, 315, 321, 327, 333, 349, 357, 359, 361, 367, 373, 389, 397, 405, 413, 437, 469, 489, 507, 509, 521, 525, 529, 537, 545, 549, 551, 565, 600, 611, 635, 654, 656, 683, 685, 699, 718, 742, 756, 803, 819, 839, 855, 865, 867, 869, 902, 905, 907, 908, 931, 937, 963, 990, 999, 1009, 1011, 1019, 1021, 1033, 1040, 1076, 1080, 1086, 1093, 1094, 1096, 1098, 1102, 1110, 1127, 1143, 1177, 1215, 1219, 1221, 1229, 1243, 1264, 1266, 1284, 1292, 1301, 1303, 1315], [3, 7, 19, 53, 63, 77, 87, 99, 111, 125, 129, 150, 152, 154, 168, 174, 178, 223, 249, 251, 255, 285, 308, 326, 332, 344, 352, 374, 380, 448, 458, 468, 470, 474, 500, 514, 518, 520, 522, 538, 550, 554, 558, 627, 629, 640, 650, 678, 694, 698, 717, 727, 763, 780, 788, 802, 880, 882, 891, 895, 901, 934, 941, 946, 958, 968, 978, 986, 989

In [17]:
print(len(ex_dic))

1236


In [18]:
er_list_2 = get_er_ratio_in_time(dialogue_dic[0],dialogue_dic[1],dialogue_dic[2],ex_in_timeslot(ex_dic,2500),0 ,1196)

In [19]:
test_ex = ex_in_timeslot(ex_dic,2500)
print(test_ex)

{'the': [3, [0, 26, 44, 60, 62, 68, 70, 72, 78, 82, 96, 98, 114, 128, 138, 140, 155, 159, 161, 165, 167, 179, 183, 189, 193, 195, 205, 206, 222, 224, 240, 244, 246, 274, 282, 284, 286, 298, 300, 302, 306, 309, 311, 315, 321, 327, 333, 349, 357, 359, 361, 367, 373, 389, 397, 405, 413, 437, 469, 489, 507, 509, 521, 525, 529, 537, 545, 549, 551, 565, 600, 611, 635, 654, 656, 683, 685, 699, 718, 742, 756, 803, 819, 839, 855, 865, 867, 869, 902, 905, 907, 908, 931, 937, 963, 990, 999, 1009, 1011, 1019, 1021, 1033, 1040, 1076, 1080, 1086, 1093, 1094, 1096, 1098, 1102, 1110, 1127, 1143, 1177, 1215, 1219, 1221, 1229, 1243, 1264, 1266, 1284, 1292, 1301, 1303, 1315], [3, 7, 19, 53, 63, 77, 87, 99, 111, 125, 129, 150, 152, 154, 168, 174, 178, 223, 249, 251, 255, 285, 308, 326, 332, 344, 352, 374, 380, 448, 458, 468, 470, 474, 500, 514, 518, 520, 522, 538, 550, 554, 558, 627, 629, 640, 650, 678, 694, 698, 717, 727, 763, 780, 788, 802, 880, 882, 891, 895, 901, 934, 941, 946, 958, 968, 978, 986, 989

In [21]:
print(len(test_ex))

1236


In [22]:
er_list_3 = get_er_ratio_in_time(dialogue_dic[0],dialogue_dic[1],dialogue_dic[2],ex_dic,0 ,1236)

In [23]:
for x in er_list_2:
    print(x)

0.6517509727626459
0.6835965978128797
0.6667429355908935


In [24]:
for x in er_list_3:
    print(x)

0.6519290928050052
0.6887307236061685
0.6691453940066593


In [24]:
slots = number_of_slot(dialogue_dic[1],dialogue_dic[2],50)

In [25]:
print(slots)

23


In [26]:
er_result = get_er_ratio_temporal(dialogue_dic[0],dialogue_dic[1],dialogue_dic[2],50,ex_dic)

In [27]:
print(er_result)

[{'begin': 0, 'end': 50, 's1_er_ratio': 0.30095238095238097, 's2_er_ratio': 0.3958333333333333, 'er_ratio_average': 0.31561996779388085}, {'begin': 50, 'end': 100, 's1_er_ratio': 0.4472361809045226, 's2_er_ratio': 0.6226415094339622, 'er_ratio_average': 0.46153846153846156}, {'begin': 100, 'end': 150, 's1_er_ratio': 0.5726141078838174, 's2_er_ratio': 0.6774193548387096, 'er_ratio_average': 0.5895652173913043}, {'begin': 150, 'end': 200, 's1_er_ratio': 0.6329113924050633, 's2_er_ratio': 0.664179104477612, 'er_ratio_average': 0.6383812010443864}, {'begin': 200, 'end': 250, 's1_er_ratio': 0.641213389121339, 's2_er_ratio': 0.5681818181818182, 'er_ratio_average': 0.638}, {'begin': 250, 'end': 300, 's1_er_ratio': 0.6767317939609236, 's2_er_ratio': 0.7228915662650602, 'er_ratio_average': 0.6826625386996904}, {'begin': 300, 'end': 350, 's1_er_ratio': 0.6490566037735849, 's2_er_ratio': 0.7590361445783133, 'er_ratio_average': 0.6639477977161501}, {'begin': 350, 'end': 400, 's1_er_ratio': 0.69675

In [29]:
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles/output_er_summary_dialign'
dl = []
import glob,os
files = glob.glob('S***.tsv')
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers(dff)
    dialogue_dic_f = extract_dialogues_dic(speaker_f[0],speaker_f[1])
    ex_dic_f = get_er_dic(dialogue_dic_f[0],dialogue_dic_f[1],dialogue_dic_f[2])
#     for k,v in ex_dic_f.items():
#         print(f,k,',',v)
#     print(dialogue_dic_f[1][4])
    er_result_f = get_er_ratio_temporal(dialogue_dic_f[0],dialogue_dic_f[1],dialogue_dic_f[2],50,ex_dic_f)
    for fx in er_result_f:
        fx['file'] = f
    
    dl_s = pd.DataFrame(er_result_f)
    dl_s.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)

In [147]:
list1 = ['hello','love','life']
c = list1.count('l')
print(c)

0


[]


IndexError: list index out of range